In [1]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('album.csv','album'),
    ('album2.csv', 'album2'),
    ('artist.csv','artist'),
    ('customer.csv','customer'),
    ('employee.csv', 'employee'),
    ('genre.csv','genre'),
    ('invoice.csv', 'invoice'),
    ('invoice_line.csv', 'invoice_line'),
    ('media_type.csv','media_type'),
    ('playlist.csv','playlist'),
    ('playlist_track.csv', 'playlist_track'),
    ('track.csv','track')
    
 # Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'avro1234',
    database = 'music_data'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = "C:/Users/AVRODITA/OneDrive/Desktop/SQL/music store data"

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing album.csv
NaN values before replacement:
album_id     0
title        0
artist_id    0
dtype: int64

Processing album2.csv
NaN values before replacement:
album_id     0
title        0
artist_id    0
dtype: int64

Processing artist.csv
NaN values before replacement:
artist_id    0
name         0
dtype: int64

Processing customer.csv
NaN values before replacement:
customer_id        0
first_name         0
last_name          0
company           49
address            0
city               0
state             29
country            0
postal_code        4
phone              1
fax               47
email              0
support_rep_id     0
dtype: int64

Processing employee.csv
NaN values before replacement:
employee_id    0
last_name      0
first_name     0
title          0
reports_to     1
levels         0
birthdate      0
hire_date      0
address        0
city           0
state          0
country        0
postal_code    0
phone          0
fax            0
email          0
dtype: int6

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector


db = mysql.connector.connect( host = 'localhost',
                              user = 'root',
                              password = 'avro1234',
                              database = 'music_data')
cur = db.cursor()

# Who is the senior most employee based on job title?

In [3]:
query =  """SELECT 
    *
FROM
    employee
ORDER BY levels DESC
LIMIT 1 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9,Madan,Mohan,Senior General Manager,None,L7,26-01-1961 00:00,14-01-2016 00:00,1008 Vrinda Ave MT,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,madan.mohan@chinookcorp.com


# Which countries have the most Invoices?

In [17]:
query="""SELECT 
    COUNT(*) AS C, billing_country
FROM
    invoice
GROUP BY billing_country
ORDER BY C DESC"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["invoice","billing_country"])
df.head()


,invoice,billing_country
0,131,USA
1,76,Canada
2,61,Brazil
3,50,France
4,41,Germany


# What are top 3 values of total invoice?

In [19]:
query="""SELECT 
    total
FROM
    invoice
ORDER BY total DESC
LIMIT 3"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["total_invoice"])
df.head()


,total_invoice
0,23.76
1,19.80
2,19.80


# Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals

In [36]:
query="""SELECT 
    SUM(total) AS invoice_total, billing_city
FROM
    invoice
GROUP BY billing_city
ORDER BY invoice_total DESC"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["invoice_total","billing_city"])
df.head(10)

,invoice_total,billing_city
0,273.240000,Prague
1,169.290001,Mountain View
2,166.319999,London
3,158.399999,Berlin
4,151.470001,Paris
5,129.690000,São Paulo
6,114.839999,Dublin
7,111.870000,Delhi
8,108.899999,São José dos Campos
9,106.920000,Brasília


# Who is the best customer? The customer who has spent the most money will be declared the best customer. Write a query that returns the person who has spent the most money

In [21]:
query="""SELECT 
    customer.customer_id,
    customer.first_name,
    customer.last_name,
    SUM(invoice.total) AS total_spending
FROM
    customer
        JOIN
    invoice ON customer.customer_id = invoice.customer_id
GROUP BY customer.customer_id
ORDER BY total_spending DESC
LIMIT 1"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["customer_id","first_name","last_name","total_spending"])
df.head()

,customer_id,first_name,last_name,total_spending
0,5,František,Wichterlová,144.539998


# Write query to return the email, first name, last name, & Genre of all Rock Music listeners.Return your list ordered alphabetically by email starting with A

In [37]:
query="""SELECT DISTINCT email,first_name, last_name
FROM customer
JOIN invoice ON customer.customer_id = invoice.customer_id
JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
WHERE track_id IN(
	SELECT track_id FROM track
	JOIN genre ON track.genre_id = genre.genre_id
	WHERE genre.name LIKE 'Rock'
)
ORDER BY email
LIMIT 10;
"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["email","first_name","last_name"])
df.head(10)

,email,first_name,last_name
0,aaronmitchell@yahoo.ca,Aaron,Mitchell
1,alero@uol.com.br,Alexandre,Rocha
2,astrid.gruber@apple.at,Astrid,Gruber
3,bjorn.hansen@yahoo.no,Bjørn,Hansen
4,camille.bernard@yahoo.fr,Camille,Bernard
5,daan_peeters@apple.be,Daan,Peeters
6,diego.gutierrez@yahoo.ar,Diego,Gutiérrez
7,dmiller@comcast.com,Dan,Miller
8,dominiquelefebvre@gmail.com,Dominique,Lefebvre
9,edfrancis@yachoo.ca,Edward,Francis


# Let's invite the artists who have written the most rock music in our dataset.Write a query that returns the Artist name and total track count of the top 10 rock bands

In [38]:
query="""SELECT artist.artist_id, artist.name, COUNT(artist.artist_id) AS number_of_songs
FROM track
JOIN album ON album.album_id = track.album_id
JOIN artist ON artist.artist_id = album.artist_id
JOIN genre ON genre.genre_id = track.genre_id
WHERE genre.name LIKE 'Rock'
GROUP BY artist.artist_id
ORDER BY number_of_songs DESC
LIMIT 10"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["artist_id","name","number_of_songs"])
df.head(10)

,artist_id,name,number_of_songs
0,22,Led Zeppelin,114
1,150,U2,112
2,58,Deep Purple,92
3,90,Iron Maiden,81
4,118,Pearl Jam,54
5,152,Van Halen,52
6,51,Queen,45
7,142,The Rolling Stones,41
8,76,Creedence Clearwater Revival,40
9,52,Kiss,35


# Return all the track names that have a song length longer than the average song length.Return the Name and Milliseconds for each track. Order by the song length with the longest songs listed first.

In [25]:
query="""SELECT name,milliseconds
FROM track
WHERE milliseconds > (
	SELECT AVG(milliseconds) AS avg_track_length
	FROM track )
ORDER BY milliseconds DESC"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["name","milliseconds"])
df.head()

,name,milliseconds
0,Occupation / Precipice,5286953
1,Through a Looking Glass,5088838
2,"Greetings from Earth, Pt. 1",2960293
3,The Man With Nine Lives,2956998
4,"Battlestar Galactica, Pt. 2",2956081


# Find how much amount spent by each customer on artists? Write a query to return customer name, artist name and total spent

In [26]:
query="""WITH best_selling_artist AS (
	SELECT artist.artist_id AS artist_id, artist.name AS artist_name, SUM(invoice_line.unit_price*invoice_line.quantity) AS total_sales
	FROM invoice_line
	JOIN track ON track.track_id = invoice_line.track_id
	JOIN album ON album.album_id = track.album_id
	JOIN artist ON artist.artist_id = album.artist_id
	GROUP BY 1
	ORDER BY 3 DESC
	LIMIT 1
)
SELECT c.customer_id, c.first_name, c.last_name, bsa.artist_name, SUM(il.unit_price*il.quantity) AS amount_spent
FROM invoice i
JOIN customer c ON c.customer_id = i.customer_id
JOIN invoice_line il ON il.invoice_id = i.invoice_id
JOIN track t ON t.track_id = il.track_id
JOIN album alb ON alb.album_id = t.album_id
JOIN best_selling_artist bsa ON bsa.artist_id = alb.artist_id
GROUP BY 1,2,3,4
ORDER BY 5 DESC;"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["customer_id","first_name","last_name","artist_name","amount_spent"])
df.head()

,customer_id,first_name,last_name,artist_name,amount_spent
0,46,Hugh,O'Reilly,Queen,27.72
1,38,Niklas,Schröder,Queen,18.81
2,3,François,Tremblay,Queen,17.82
3,34,João,Fernandes,Queen,16.83
4,53,Phil,Hughes,Queen,11.88


# We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases. Write a query that returns each country along with the top Genre.For countries where the maximum number of purchases is shared return all Genres

In [29]:
query="""   ( SELECT COUNT(invoice_line.quantity) AS purchases, customer.country, genre.name, genre.genre_id, 
	ROW_NUMBER() OVER(PARTITION BY customer.country ORDER BY COUNT(invoice_line.quantity) DESC) AS RowNo 
    FROM invoice_line 
	JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
	JOIN customer ON customer.customer_id = invoice.customer_id
	JOIN track ON track.track_id = invoice_line.track_id
	JOIN genre ON genre.genre_id = track.genre_id
	GROUP BY 2,3,4
	ORDER BY 2 ASC, 1 DESC
)"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["purchase","country","name","genre_id","row_no."])
df.head()

,purchase,country,name,genre_id,row_no.
0,17,Argentina,Alternative & Punk,4,1
1,11,Argentina,Rock,1,2
2,2,Argentina,Blues,6,3
3,2,Argentina,Metal,3,4
4,2,Argentina,Latin,7,5


# Write a query that determines the customer that has spent the most on music for each country.Write a query that returns the country along with the top customer and how much they spent.For countries where the top amount spent is shared, provide all customers who spent this amount.

In [30]:
query="""WITH Customter_with_country AS (
		SELECT customer.customer_id,first_name,last_name,billing_country,SUM(total) AS total_spending,
	    ROW_NUMBER() OVER(PARTITION BY billing_country ORDER BY SUM(total) DESC) AS RowNo 
		FROM invoice
		JOIN customer ON customer.customer_id = invoice.customer_id
		GROUP BY 1,2,3,4
		ORDER BY 4 ASC,5 DESC)
SELECT * FROM Customter_with_country WHERE RowNo <= 1"""

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns=["customer_id","first_name","last_name","billing_country","total_spending","row_no"])
df.head()

,customer_id,first_name,last_name,billing_country,total_spending,row_no
0,56,Diego,Gutiérrez,Argentina,39.599999,1
1,55,Mark,Taylor,Australia,81.180000,1
2,7,Astrid,Gruber,Austria,69.299999,1
3,8,Daan,Peeters,Belgium,60.390000,1
4,1,Luís,Gonçalves,Brazil,108.899999,1
